In [1]:

import functools
import traceback


def log_exceptions(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        func_name = func.__name__
        try:
            result = func(*args, **kwargs)
        except Exception as e:
            print(f"FAILURE: {func_name} failed: {e}")
            traceback.print_exc()
        else:
            print(f"SUCCESS: {func_name} succeeded")
            return result
    return wrapper


def decorate_methods(decorator):
    def decorate(obj):
        if isinstance(obj, type):
            # Decorate class methods
            for name, method in vars(obj).items():
                if callable(method):
                    setattr(obj, name, decorator(method))
            return obj
        else:
            # Decorate standalone functions
            return decorator(obj)
    return decorate


%load_ext autoreload
%autoreload 2

In [6]:
import os 
import sys 
import docker


# path to this file's directory and parent directory
current_directory = os.getcwd()
base_directory = os.path.dirname(current_directory)

# parent directory of base_directory
parent_directory = os.path.dirname(base_directory)

# add to path
sys.path.append(base_directory)

# define paths to each app dockerfile location 
processor_path = os.path.join(base_directory, 'quick_batch', 'processor_app')
controller_path = os.path.join(base_directory, 'quick_batch', 'controller_app')
queue_path = os.path.join(base_directory, 'quick_batch', 'queue_app')
config_path = os.path.join(base_directory, 'tests/test_configs', 'test_quick_batch.yaml')
input_path = os.path.join(base_directory, 'tests/test_data', 'input_data')
output_path = os.path.join(base_directory, 'tests/test_data', 'output_data')

print(f'base_directory: {base_directory}')
print(f'processor_path: {processor_path}')
print(f'controller_path: {controller_path}')
print(f'queue_path: {queue_path}')
print(f'config_path: {config_path}')
print(f'input_path: {input_path}')
print(f'output_path: {output_path}')



base_directory: /Users/wattjer/Desktop/quick_batch
processor_path: /Users/wattjer/Desktop/quick_batch/quick_batch/processor_app
controller_path: /Users/wattjer/Desktop/quick_batch/quick_batch/controller_app
queue_path: /Users/wattjer/Desktop/quick_batch/quick_batch/queue_app
config_path: /Users/wattjer/Desktop/quick_batch/tests/test_configs/test_quick_batch.yaml
input_path: /Users/wattjer/Desktop/quick_batch/tests/test_data/input_data
output_path: /Users/wattjer/Desktop/quick_batch/tests/test_data/output_data


# services and swarm

In [188]:
@log_exceptions
def create_client():
    return docker.from_env()

@log_exceptions
def remove_all_containers(client):
    for container in client.containers.list(all=True):
        container.remove(force=True)

@log_exceptions
def create_network(client):
    if 'quick_batch_network' not in [network.name for network in client.networks.list()]:
        client.networks.create('quick_batch_network', driver='overlay')
        
@log_exceptions
def remove_network(client):
    if 'quick_batch_network' in [network.name for network in client.networks.list()]:
        client.networks.get('quick_batch_network').remove()

@log_exceptions
def leave_swarm(client):
    client.swarm.leave(force=True)
        
@log_exceptions
def create_swarm(client):
    client.swarm.init()

@log_exceptions
def remove_service(client, service_name):
    if service_name in [service.name for service in client.services.list()]:
        client.services.get(service_name).remove()
        
@log_exceptions
def remove_all_services(client):
    for service in client.services.list():
        service.remove()
 

@log_exceptions
def create_queue_service(client):
    # remove queue_app service if it exists
    remove_service(client, 'queue_app')
    
    # Define the mounts for the service containers
    mounts = [
        docker.types.Mount(
            type='bind',
            source=queue_path + '/queue_app',
            target='/queue_app',
            read_only=True
        ),
        docker.types.Mount(
            type='bind',
            source=config_path,
            target='/my_configs/config.yaml',
            read_only=True
        ),
        docker.types.Mount(
            type='bind',
            source=input_path,
            target='/my_data/input',
            read_only=True
        ),
    ]
    
    # Define the service configuration
    service_config = {
        'image': 'quick_batch_queue_app',
        'name': 'queue_app',
        'maxreplicas': 1,
        'log_driver': 'json-file',
        'log_driver_options': {'max-size': '10m', 'max-file': '3'},
        'restart_policy': {'Condition': 'none', 'MaxAttempts': 0},
        'mounts': mounts,
        'networks':['quick_batch_network'],
        'command': ['python', '/queue_app/run.py']
    }

    # Create the service
    service = client.services.create(**service_config)
    return service


@log_exceptions
def create_processor_service(client):
    remove_service(client, 'processor_app')

    # Define the mounts for the service containers
    mounts = [
        docker.types.Mount(
            type='bind',
            source=processor_path + '/processor_app',
            target='/processor_app',
            read_only=True
        ),
        docker.types.Mount(
            type='bind',
            source='/Users/wattjer/Desktop/quick_batch/processor.py',
            target='/custom_processor/processor.py',
            read_only=False
        ),    
        docker.types.Mount(
            type='bind',
            source=config_path,
            target='/my_configs/config.yaml',
            read_only=True
        ),
        docker.types.Mount( 
            type='bind',
            source=input_path,
            target='/my_data/input',
            read_only=True
        ),
        docker.types.Mount(
            type='bind',
            source=output_path,
            target='/my_data/output',
            read_only=False
        ),
    ]

    # Define the service configuration
    service_config = {
        'image': 'quick_batch_processor_app',
        'name': 'processor_app',
        'log_driver': 'json-file',
        'log_driver_options': {'max-size': '10m', 'max-file': '3'},
        'restart_policy': {'Condition': 'none', 'MaxAttempts': 0},
        'mounts': mounts,
        'networks':['quick_batch_network'],
        'command': ['python', '/processor_app/run.py'],
        # 'command': ['tail', '-f', '/dev/null'],
    }

    service = client.services.create(**service_config)
    return service

import json 

def get_current_queue_lengths(client):
    # Get the list of running containers for the service
    queue_app_container = client.containers.list(filters={'label': 'com.docker.swarm.service.name=queue_app'})

    # Assuming there is only one container for the service
    if queue_app_container:
        queue_app_container = queue_app_container[0]
        command = 'curl -s localhost:80'
        response = queue_app_container.exec_run(command)
        response = json.loads(response.output.decode('utf-8'))
        return response
    else:
        print("No running containers found for the service.")
        return None
        


In [183]:
# creater docker client
client = docker.from_env()

# remove all services 
remove_all_services(client)

SUCCESS: remove_all_services succeeded


In [184]:
client.services.list()

[]

In [185]:
import time 


# creater docker client
client = docker.from_env()

# remove all services 
remove_all_services(client)
time.sleep(5)

# remove swarms 
leave_swarm(client)

# create new swarm
create_swarm(client)

# create network
create_network(client)
time.sleep(5)

# remove all services
remove_all_services(client)

# create queue service
queue_service = create_queue_service(client)

# create processor service
processor_service = create_processor_service(client)
scale = processor_service.scale(2)

# watch feed_queue_length, when it reaches 0, stop the processor service
time.sleep(5)
while True:
    response = get_current_queue_lengths(client)
    queue_lengths = get_current_queue_lengths(client)
    feed_queue_length = response['feed_queue_length']
    print(feed_queue_length, flush=True)
    if feed_queue_length == 0:
     
        remove_all_services(client)
        remove_all_containers(client)
        remove_network(client)
        
        break
    time.sleep(5)

SUCCESS: remove_all_services succeeded
SUCCESS: leave_swarm succeeded
SUCCESS: create_swarm succeeded
SUCCESS: create_network succeeded
SUCCESS: remove_all_services succeeded
SUCCESS: remove_service succeeded
SUCCESS: create_queue_service succeeded
SUCCESS: remove_service succeeded
SUCCESS: create_processor_service succeeded
10
10
10
8
8
8
6
6
6
4
4
3
2
2
1
0
